In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [18]:
btc_usd = pd.read_csv("BTC-USD.csv")
btc_usd = btc_usd.loc[:,["Date","Close"]]
btc_cad = pd.read_csv("BTC-CAD.csv")
btc_cad = btc_cad.loc[:,["Date","Close"]]
btc_eur = pd.read_csv("BTC-EUR.csv")
btc_eur = btc_eur.loc[:,["Date","Close"]]
btc_gbp = pd.read_csv("BTC-GBP.csv")
btc_gbp = btc_gbp.loc[:,["Date","Close"]]
eur_cad = pd.read_csv("EUR-CAD.csv")
eur_cad = eur_cad.loc[:,["Date","Close"]]
eur_gbp = pd.read_csv("EUR-GBP.csv")
eur_gbp = eur_gbp.loc[:,["Date","Close"]]
eur_usd = pd.read_csv("EUR-USD.csv")
eur_usd = eur_usd.loc[:,["Date","Close"]]
gbp_cad = pd.read_csv("GBP-CAD.csv")
gbp_cad = gbp_cad.loc[:,["Date","Close"]]
gbp_usd = pd.read_csv("GBP-USD.csv")
gbp_usd = gbp_usd.loc[:,["Date","Close"]]
cad_usd = pd.read_csv("CAD-USD.csv")
cad_usd = cad_usd.loc[:,["Date","Close"]]

Section below will look at the influence of triangular arbitrage on cross-currency correlation. We will mainly focus on three currencies: USD, GBP, and BTC.

In [33]:
same = 0
for i in range(len(btc_usd["Close"])-1):
    if (btc_usd["Close"][i] < btc_usd["Close"][i + 1]) and (btc_gbp["Close"][i] < btc_gbp["Close"][i + 1]):
        same = same + 1
    elif (btc_usd["Close"][i] > btc_usd["Close"][i + 1]) and (btc_gbp["Close"][i] > btc_gbp["Close"][i + 1]):
        same = same + 1
same/len(btc_usd["Close"])

0.9754098360655737

In [65]:
same = 0
for i in range(len(gbp_usd["Close"])-1):
    if (btc_usd["Close"][i] <= btc_usd["Close"][i + 1]) and (gbp_usd["Close"][i] <= gbp_usd["Close"][i + 1]):
        same = same + 1
    elif (btc_usd["Close"][i] >= btc_usd["Close"][i + 1]) and (gbp_usd["Close"][i] >= gbp_usd["Close"][i + 1]):
        same = same + 1
same/len(btc_usd["Close"])

0.6557377049180327

In [68]:
same = 0
for i in range(len(gbp_usd["Close"])-1):
    if (btc_gbp["Close"][i] <= btc_gbp["Close"][i + 1]) and (gbp_usd["Close"][i] <= gbp_usd["Close"][i + 1]):
        same = same + 1
    elif (btc_gbp["Close"][i] >= btc_gbp["Close"][i + 1]) and (gbp_usd["Close"][i] >= gbp_usd["Close"][i + 1]):
        same = same + 1
1-same/len(btc_usd["Close"])

0.3415300546448088